# Changing to Parent Directory to Import Needed Libraries

In [2]:
import os

# Change this relative to your own directory structure
PARENT_DIR = '/Users/henrygilbert/GitHub/CS-6362/final_project'
os.chdir(PARENT_DIR)

# Importing Needed Libraries

In [24]:
import torch
import numpy as np
from typing import Tuple, List
import pandas as pd
import importlib
import copy
from torch.utils.data import DataLoader

import utilities
import factor_data_loader
import market_data_loader
from model import CVAE
from matplotlib import pyplot as plt

importlib.reload(utilities)


<module 'utilities' from '/Users/henrygilbert/GitHub/CS-6362/final_project/utilities.py'>

# Loading Data 

In [42]:


mdl = market_data_loader.MarketDataLoader()
fdl = factor_data_loader.FactorDataLoader()

start_ts = pd.Timestamp('2016-01-01')
end_ts = pd.Timestamp('2021-02-01')

monthly_eod_prices = mdl.get_eod_price_data_grouped('SPY', start_ts, end_ts, market_data_loader.GroupPeriod.MONTHLY)
all_eod_prices = np.concatenate([prices for prices in list(monthly_eod_prices.values())])

percent_change = np.diff(all_eod_prices)/all_eod_prices[:-1]
std_change = np.std(percent_change)
mean_change = np.mean(percent_change)

print(f"Mean: {mean_change}, Std: {std_change}")

week_size = 5
weekly_data = [percent_change[i:i+week_size] for i in range(0, len(percent_change), week_size)]

# only condition on previous week, no external factor
weekly_trainng_data = [
    (weekly_data[i-1], weekly_data[i]) 
    for i in range(1, len(weekly_data))
    if len(weekly_data[i]) == 5 and len(weekly_data[i-1]) == 5]

month_batch_size = 4
monthly_batches = [
    weekly_trainng_data[i:i+month_batch_size] 
    for i in range(0, len(weekly_trainng_data), month_batch_size)]


/Users/henrygilbert/GitHub/CS-6362/final_project/market_data_loader.py:66: UserWarning: Converting to Period representation will drop timezone information.
  group.index[0].to_period(group_by.value).to_timestamp(): group['close'].to_numpy()


# Training/Evaluation

In [39]:
batch_size = 4
batch_to_rmse = {}

for i in range(len(monthly_batches)):
    
    cvae = CVAE(5, 5).to(utilities.DEVICE)
    training_batches = copy.deepcopy(monthly_batches)
    test_batch = training_batches.pop(i)
    
    training_weeks = [week for batch in training_batches for week in batch]
    training_data = utilities.ConditionedMarketDataset(training_weeks)
    testing_data = utilities.ConditionedMarketDataset(test_batch)
   
    train_dataset = DataLoader(training_data, batch_size=batch_size, shuffle=True)
    testing_dataset = DataLoader(testing_data, batch_size=batch_size, shuffle=True)
    
    history = utilities.train_model(cvae, train_dataset, testing_dataset, epochs=10)
    utilities.save_loss(history, "loss.png")
    
    predicted_val_prices = []
    predicted_synthetic_prices = []
    actual_val_prices = []
    
    for batch in testing_dataset:
        
        price_batch = batch['price_data']
        print(price_batch)
        continue
        synthetic_price_batches = [torch.FloatTensor(np.array([np.random.randn(len(b)) for b in price_batch])) for _ in range(100)]
        
        conditioned_batch = batch['factor_data']
        price_batch = price_batch.to(utilities.DEVICE)
        
        predicted_prices = cvae(price_batch.float(), conditioned_batch.float())
        predicted_val_prices += (predicted_prices*std_price+mean_price).detach().numpy().tolist()
        actual_val_prices += (price_batch*std_price+mean_price).detach().numpy().tolist()
       
        sample_synthetic_prices = [cvae(synthetic_b.float(), conditioned_batch.float()).detach().numpy() for synthetic_b in synthetic_price_batches]
        mean_synthetic_prices = np.mean(sample_synthetic_prices, axis=0)
        predicted_synthetic_prices += (mean_synthetic_prices*std_price+mean_price).tolist()
    
    batch_to_rmse[i] = utilities.rmse_loss_fn(
        torch.FloatTensor(np.array(predicted_val_prices)), 
        torch.FloatTensor(np.array(actual_val_prices)))
    
    plt.plot(np.array(predicted_synthetic_prices).flatten(), label="synthetic mean over 100 samples")
    plt.plot(np.array(actual_val_prices).flatten(), label="historical realization")
    plt.legend()
    plt.xlabel("week number")
    plt.ylabel("price in USD")
    plt.savefig(f"experiments/experiment_1/graphs/synthetic_data_prediction_batch{i}.png")
    plt.clf()
    
plt.bar(batch_to_rmse.keys(), batch_to_rmse.values())
plt.xlabel("Test Batch Number")
plt.ylabel("RMSE between averaged synthetic and actual prices")
plt.savefig(f"experiments/experiment_1/graphs/rmse_{i}.png")
plt.clf()
    


tensor([[-1.7620, -1.7269, -1.8363, -1.7661, -1.8594],
        [-1.8537, -1.9083, -1.8848, -1.7980, -1.8632],
        [-1.8053, -1.8521, -1.8300, -1.7255, -1.7271],
        [-1.8062, -1.7804, -1.7736, -1.8562, -1.9135]], dtype=torch.float64)


<Figure size 1500x900 with 0 Axes>

tensor([[-1.7446, -1.7625, -1.7645, -1.7015, -1.7573],
        [-1.9133, -1.9169, -1.9715, -1.8861, -1.8148],
        [-1.6060, -1.5884, -1.5736, -1.5700, -1.6196],
        [-1.7373, -1.6843, -1.6945, -1.7339, -1.6261]], dtype=torch.float64)


<Figure size 1500x900 with 0 Axes>

tensor([[-1.4583, -1.4266, -1.4418, -1.4885, -1.4380],
        [-1.5077, -1.4785, -1.4841, -1.4776, -1.4801],
        [-1.5107, -1.5127, -1.5100, -1.4674, -1.4470],
        [-1.5974, -1.5938, -1.5208, -1.5267, -1.5342]], dtype=torch.float64)


...

tensor([[-0.5930, -0.5978, -0.6016, -0.5761, -0.5810],
        [-0.6028, -0.5838, -0.5876, -0.6039, -0.5944],
        [-0.6648, -0.6381, -0.6383, -0.6429, -0.6453],
        [-0.7697, -0.7509, -0.7167, -0.6884, -0.6768]], dtype=torch.float64)


<Figure size 1500x900 with 0 Axes>

tensor([[-0.6134, -0.6059, -0.6037, -0.6465, -0.5989],
        [-0.6299, -0.6243, -0.6281, -0.5883, -0.5774],
        [-0.5921, -0.6177, -0.5722, -0.6082, -0.6109],
        [-0.6460, -0.6370, -0.6250, -0.6155, -0.6644]], dtype=torch.float64)


KeyboardInterrupt: 

<Figure size 1500x900 with 0 Axes>